In [1]:
import numpy as np
import cv2

In [6]:
with_mask=np.load('Desktop/Jupyter/Face-Mask-Detection/Face-Mask-Detection-master/dataset/with_mask')
without_mask=np.load('Desktop/Jupyter/Face-Mask-Detection/Face-Mask-Detection-master/dataset/without_mask')

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/Jupyter/Face-Mask-Detection/Face-Mask-Detection-master/dataset/without_mask/0_0_aidai_0014.jpg'

In [ ]:
without_mask.shape

In [ ]:
with_mask=with_mask.reshape(200,50*50*3)
without_mask=without_mask.reshape(200,50*50*3)

In [ ]:
with_mask.shape

In [33]:
without_mask.shape

(200, 7500)

In [34]:
X=np.r_[with_mask,without_mask]

In [35]:
X.shape

(400, 7500)

In [36]:
labels=np.zeros(X.shape[0])

In [37]:
labels[200:]=1.0

In [94]:
names={0:'Mask',1:'No Mask'}

In [95]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [96]:
from sklearn.svm import SVC

In [97]:
from sklearn.model_selection import train_test_split

In [98]:
x_train,x_test,y_train,y_test=train_test_split(X,labels,test_size=0.23)

In [99]:
x_train.shape

(308, 7500)

In [100]:
from sklearn.decomposition import PCA

In [101]:
pca=PCA(n_components=3)
x_train=pca.fit_transform(x_train)

In [102]:
x_train[0]

array([-1090.11372976,  3083.05932565,  -762.18071703])

In [103]:
x_train.shape

(308, 3)

In [104]:
svm=SVC()
svm.fit(x_train, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [105]:
x_test=pca.transform(x_test)
y_pred=svm.predict(x_test)

In [106]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9565217391304348


In [112]:
haar_data=cv2.CascadeClassifier('face.xml')
capture = cv2.VideoCapture(0)
data=[]
font=cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = haar_data.detectMultiScale(gray)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x,y),(x+w,y+h),(255,0,0),4)
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (50, 50))
            face = face.reshape(1, -1)  # Flatten for PCA
            face_pca = pca.transform(face)
            pred = svm.predict(face_pca)
            n = names[int(pred[0])]
            cv2.putText(img,n,(x,y),font,1,(255,0,0),2)

        cv2.imshow('result', img)
        if cv2.waitKey(2) == 27:
            break
capture.release()
cv2.destroyAllWindows()
